# TP : Clustering avec DBSCAN

## 1. Introduction

### 🌟 Objectifs du TP
- Comprendre le fonctionnement de l'algorithme DBSCAN.
- L'utiliser pour regrouper des données sans connaître à l'avance le nombre de clusters.
- Comparer ses résultats à ceux de K-Means.

---

## 2. Théorie : DBSCAN

### 🔍 Qu'est-ce que DBSCAN ?
DBSCAN (Density-Based Spatial Clustering of Applications with Noise) est un algorithme de clustering basé sur la densité des points.
Il est capable de trouver des clusters de formes arbitraires et de reconnaître les points isolés comme du bruit.

### ⚙️ Paramètres importants :
- `eps` : **distance maximale** pour qu’un point soit considéré comme voisin.
- `min_samples` : **nombre minimum de points** dans un voisinage pour former un cluster.

### 🧹 Types de points :
- Point **central** : assez de voisins proches.
- Point **bordure** : proche d’un point central mais sans assez de voisins.
- Point **bruit** : trop éloigné des autres.

### 🔍 Attribution de label :
Le modèle DBSCAN attribue à chaque point :

- soit un **numéro de cluster** (ex. 0, 1, 2…),

- soit -1 pour les points considérés comme du **bruit** (outliers).



## 3. Avantages et inconvénients de DBSCAN

### ✅ Avantages :
- Identifie des clusters de **forme irrégulière**.
- Gère automatiquement les **points aberrants** (bruit).
- Pas besoin de préciser le nombre de clusters.

### ❌ Inconvénients :
- Sensible au choix des paramètres `eps` et `min_samples`.
- Moins efficace si la densité varie beaucoup entre clusters.

### ℹ️ Note pédagogique – Post-traitement possible :
Une fois les clusters détectés par DBSCAN, il est parfois utile de :

✅ **Supprimer les clusters trop petits** considérés comme peu significatifs.

🔁 Ou **tenter de les fusionner** avec des clusters proches.

💡 *Ces étapes relèvent du post-traitement du clustering,
elles sont utiles mais ne sont pas abordées dans ce TP.*

---

## 4. Expérimentation sur un jeu de données simulé

### 📁 4.1 Génération de données

In [ ]:
# Installation des bibliothèques
!pip install pandas numpy scikit-learn matplotlib

In [ ]:
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
import numpy as np

X, y = make_moons(n_samples=300, noise=0.05, random_state=42)
plt.scatter(X[:, 0], X[:, 1], s=50)
plt.title("Données simulées (formes en croissant)")
plt.show()

### 🔌 4.2 Application de DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.2, min_samples=5)
labels = dbscan.fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='plasma', s=50)
plt.title("Clustering DBSCAN")
plt.show()

### 📊 4.3 Comparaison avec K-Means

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
kmeans.fit(X)
k_labels = kmeans.labels_

plt.scatter(X[:, 0], X[:, 1], c=k_labels, cmap='viridis', s=50)
plt.title("Clustering K-Means")
plt.show()

---

## 📘 5. Estimation automatique des paramètres DBSCAN

Dans cette section, nous allons estimer automatiquement les paramètres `min_samples` et `eps` à partir des données. 

**<u>Important</u>** : une règle *empirique* indique une fourchette entre **2 et 5 * nombre de critères** (3 * nombre de critères s'ils sont nombreux).


### 🔄 5.1 Détermination du couple eps / min_samples :

Ce code permet de tester plusieurs combinaisons de paramètres pour l’algorithme DBSCAN.

Il affiche un tableau récapitulatif des valeurs estimées pour :

- min_samples (nombre minimum de voisins)

- eps (distance maximale entre voisins)

Ensuite, il applique DBSCAN avec chaque couple (min_samples, eps)

Il affiche :

- le **nombre de clusters détectés**

- le **nombre de points** considérés comme **bruit**

- une visualisation claire des résultats

🎯 L’objectif est de comparer les résultats pour choisir le **meilleur couple de paramètres**.



In [ ]:
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import numpy as np

# Calcul de la dimension n (nombre de critères ici)
dimension = X.shape[1]

# Règle empirique pour déterminer l'intervalle de min_samples (seulement 2 dimensions ici)
min_samples_range = range(2, 5 * dimension)
eps_results = []  # Stocker les distances moyennes des plus grands écarts visuels

# Test pour chaque valeur de min_samples : le KNN permet d'estimer l'EPS
for min_samples in min_samples_range:
    neighbors = NearestNeighbors(n_neighbors=min_samples).fit(X)
    distances, _ = neighbors.kneighbors(X)
    k_distances = np.sort(distances[:, min_samples - 1])

    # En guise d'estimation simple, on prend la valeur au 90e percentile
    eps_est = np.percentile(k_distances, 90)
    eps_results.append((min_samples, round(eps_est, 3)))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


# Création du tableau à partir des résultats
eps_df = pd.DataFrame(eps_results, columns=["min_samples", "eps estimé"])
print(eps_df)

# Affichage sous forme de tableau matplotlib
fig, ax = plt.subplots()
ax.axis('off')
table = ax.table(cellText=eps_df.values,
                 colLabels=eps_df.columns,
                 cellLoc='center',
                 loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
fig.tight_layout()
plt.title("Résumé des eps estimés")
plt.show()


# Boucle de test pour chaque couple (min_samples, eps)
from sklearn.cluster import DBSCAN

for min_s, eps in eps_results:
    dbscan = DBSCAN(eps=eps, min_samples=min_s)
    labels = dbscan.fit_predict(X)

    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)

    print(f"min_samples = {min_s}, eps = {eps:.3f}, clusters = {n_clusters}, bruit = {n_noise} points")

    plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='tab10', s=50)
    plt.title(f"DBSCAN (min_samples={min_s}, eps={eps:.3f})")
    plt.show()

On voit ici que l'on choisit **min_samples = 7** et **eps = 0.136**.

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.136, min_samples=7)
labels = dbscan.fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='plasma', s=50)
plt.title("Clustering DBSCAN")
plt.show()

### 🧾 5.2 Prédiction de l'appartenance d'une donnée à un cluster

🧠 **<u>Important</u>** : DBSCAN ne permet pas la prédiction directe (pas de .predict() 😔)
mais on peut contourner ce problème en :

- entraînant un **modèle de classification supervisée (ex : KNN en particulier)** sur les résultats de DBSCAN,

- puis en utilisant ce modèle pour prédire l’appartenance d’un nouveau point.

**<u>ATTENTION</u>** : on veillera à **supprimer les données cataloguées comme "bruit"** qui fausseraient les résultats : on les repère grâce à leur label valant -1.

Comme le savez bien sûr 😁, l'algorithme des **plus proches voisins (KNN)** dépend du nombre de voisins pris en compte.
Pour connaître leur nombre le plus adapté, on peut se servir du code suivant, indiquant son score pour chaque k-voisins sollicités.

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import numpy as np


# Données très bruitées
X, y = make_moons(n_samples=300, noise=0.25, random_state=42)


# 📌 Visualisation du nuage de points
plt.figure(figsize=(6, 4))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', s=50)
plt.title("Données make_moons avec bruit important")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.show()


# 🔍 Validation croisée sur différentes valeurs de k
scores = []
k_values = range(1, 21)

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    score = cross_val_score(knn, X, y, cv=5).mean()
    scores.append(score)

    
# 📈 Affichage des scores
plt.plot(k_values, scores, marker='o')
plt.xlabel("Nombre de voisins (k)")
plt.ylabel("Score de validation croisée")
plt.title("Choix optimal de k pour KNN (données bruitées)")
plt.grid(True)
plt.show()


# ✅ Meilleur k
best_k = k_values[np.argmax(scores)]
print(f"Meilleur k estimé : {best_k} avec un score de {max(scores):.3f}")


In [ ]:
# On suppose le modèle précédent toujours entraîné
from sklearn.neighbors import KNeighborsClassifier


# Suppression des points de bruit (-1)
X_clean = X[labels != -1]
y_clean = labels[labels != -1]


# Entraînement d'un classifieur KNN selon les "k_neighbors" sur les clusters trouvés
k_neighbors = 9   # D'après précédemment 
knn = KNeighborsClassifier(n_neighbors=k_neighbors)
knn.fit(X_clean, y_clean)


# Exemple : nouvelle donnée à prédire
point_test = np.array([[1.0, 0.0]])
cluster_pred = knn.predict(point_test)
neighbors_idx = knn.kneighbors(point_test, return_distance=False)[0]
neighbors_points = X_clean[neighbors_idx]


# Visualisation
plt.figure(figsize=(8, 5))
unique_labels = set(labels)
colors = [plt.cm.plasma(each) for each in np.linspace(0, 1, len(unique_labels))]

for label, color in zip(unique_labels, colors):
    mask = labels == label
    if label == -1:
        color = (0, 0, 0, 1)  # noir pour le bruit
        label_name = "Bruit"
    else:
        label_name = f"Cluster {label}"
    plt.scatter(X[mask, 0], X[mask, 1], s=50, c=[color], label=label_name)

    
# Affichage du point test
plt.scatter(point_test[0, 0], point_test[0, 1], c="cyan", edgecolors="green", s=200, marker="*", label=f"Point test (cluster {cluster_pred})")

# Colorier les voisins KNN
plt.scatter(neighbors_points[:, 0], neighbors_points[:, 1], edgecolors="black", facecolors="none", s=200, linewidths=2, label="Voisins KNN")


plt.title("Visualisation DBSCAN avec prédiction d'un point test")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.show()

print(f"Le point {point_test[0]} est classé dans le cluster {cluster_pred[0]}")
print(knn.predict_proba(point_test)) 

---

## 🧠 6. TP : DBSCAN sur données réelles

Voici un exemple d'application de l’algorithme **DBSCAN** à des **données clients réalistes** sur deux critères : le **salaire** et les **dépenses** 

📝 **<u>Objectif</u>** : détecter automatiquement des groupes de clients ayant des comportements similaires.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN


# Chargement des données
df_clients = pd.read_csv("donnees_clients_dbscan.csv")
X = df_clients[["Revenu_Mensuel", "Depenses_Mensuelles"]].values

# Affichage des premières lignes du jeu de données
print(df_clients.head())



# Choisir une plage de min_samples_range à tester
min_samples_range = range(2,10)
eps_results = []  # Stocker les distances moyennes des plus grands écarts visuels

# Test pour chaque valeur de min_samples : le KNN permet d'estimer l'EPS
################# A COMPLETER ####################
    



# Création du tableau à partir des résultats
eps_df = pd.DataFrame(eps_results, columns=["min_samples", "eps estimé"])
print(eps_df)

# Affichage sous forme de tableau matplotlib
fig, ax = plt.subplots()
ax.axis('off')
table = ax.table(cellText=eps_df.values,
                 colLabels=eps_df.columns,
                 cellLoc='center',
                 loc='center')
table.auto_set_font_size(False)
table.set_fontsize(10)
fig.tight_layout()
plt.title("Résumé des eps estimés")
plt.show()



# Boucle de test pour chaque couple (min_samples, eps)
for min_s, eps in eps_results:
    dbscan = DBSCAN(eps=eps, min_samples=min_s)
    labels = dbscan.fit_predict(X)

    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)

    print(f"min_samples = {min_s}, eps = {eps:.3f}, clusters = {n_clusters}, bruit = {n_noise} points")

    plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='tab10', s=50)
    plt.title(f"DBSCAN (min_samples={min_s}, eps={eps:.3f})")
    plt.show()

In [ ]:
# Détermination du meilleur k pour le KNN
scores = []
k_values = range(1, 21)

################# A COMPLETER ####################


    
# 📈 Affichage des scores
plt.plot(k_values, scores, marker='o')
plt.xlabel("Nombre de voisins (k)")
plt.ylabel("Score de validation croisée")
plt.title("Choix optimal de k pour KNN (données bruitées)")
plt.grid(True)
plt.show()


# ✅ Meilleur k
best_k = k_values[np.argmax(scores)]
print(f"Meilleur k estimé : {best_k} avec un score de {max(scores):.3f}")


In [ ]:
# Entraînement du modèle DBSCAN avec le couple eps / min_samples déterminés ci-dessus    
################# A COMPLETER ####################
dbscan = DBSCAN(eps=75.9, min_samples=2)
labels = dbscan.fit_predict(X)


# Affichage des données (et clusters)
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='plasma', s=50)
plt.title("Clustering DBSCAN")
plt.show()

    
# Suppression des points de bruit (-1)
X_clean = X[labels != -1]
y_clean = labels[labels != -1]


# Entraînement d'un classifieur KNN avec le meilleur k trouvé précédemment
################# A COMPLETER ####################


# Exemple : nouvelle donnée à prédire (à l'aide de KNN)
################# A COMPLETER ####################


# Visualisation
plt.figure(figsize=(8, 5))
unique_labels = set(labels)
colors = [plt.cm.plasma(each) for each in np.linspace(0, 1, len(unique_labels))]

for label, color in zip(unique_labels, colors):
    mask = labels == label
    if label == -1:
        color = (0, 0, 0, 1)  # noir pour le bruit
        label_name = "Bruit"
    else:
        label_name = f"Cluster {label}"
    plt.scatter(X[mask, 0], X[mask, 1], s=50, c=[color], label=label_name)

    
# Affichage du point test
plt.scatter(point_test[0, 0], point_test[0, 1], c="cyan", edgecolors="green", s=200, marker="*", label=f"Point test (cluster {cluster_pred})")

# Colorier les voisins KNN
plt.scatter(neighbors_points[:, 0], neighbors_points[:, 1], edgecolors="black", facecolors="none", s=200, linewidths=2, label="Voisins KNN")


plt.title("Visualisation DBSCAN avec prédiction d'un point test")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.show()

print(f"Le point {point_test[0]} est classé dans le cluster {cluster_pred[0]}")
print(knn.predict_proba(point_test)) 

### 📌 Travail à faire / Questions :

1. **Analyser les données** : visualiser le nuage de points et **estimer** le nombre de clusters.
2. **Choisir une plage de `min_samples` à tester**.
3. Tester les résultats de **DBSCAN** pour chaque couple (`min_samples`, `eps`).
4. ✅ Choisir les **meilleurs paramètres** `eps` / `min_samples` selon :
   - nombre de clusters détectés selon la question 2.)
   - nombre de points bruités (le moins possible)
   - cohérence des groupes
   
   
5. **Déterminer** le cluster d'appartenance d'un client gagnant 2000 euros et dépendant 1500 euros par mois.
6. Quelles **améliorations** pourrait-on apporter à ce modèle ? On observera la visualisation pour ce faire.
---

💡 **Aide** : s'inspirer de l’exemple précédent sur `make_moons`, copier-coller les cellules nécessaires en adaptant les données. On vérifiera également que le meilleur couple `min_samples / eps` est *(2 , 75.9)* et que le meilleur `k` pour le KNN est *3* ou *4*.